https://www.kaggle.com/competitions/playground-series-s3e24/data

In [61]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB 
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.compose import make_column_selector, make_column_transformer

import warnings
warnings.filterwarnings('ignore')

In [62]:
smookers_df = pd.read_csv(r'C:\Users\DAI.STUDENTSDC\Desktop\Machine Learning\Data Sets\Smookers\train.csv', index_col=0)
smookers_df.head()

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,relaxation,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
id,,,,,,,,,,,,,,,,,,,,,
0,55,165,60,81.0,0.5,0.6,1,1,135,87,...,40,75,16.5,1,1.0,22,25,27,0,1
1,70,165,65,89.0,0.6,0.7,2,2,146,83,...,57,126,16.2,1,1.1,27,23,37,1,0
2,20,170,75,81.0,0.4,0.5,1,1,118,75,...,45,93,17.4,1,0.8,27,31,53,0,1
3,35,180,95,105.0,1.5,1.2,1,1,131,88,...,38,102,15.9,1,1.0,20,27,30,1,0
4,30,165,60,80.5,1.5,1.0,1,1,121,76,...,44,93,15.4,1,0.8,19,13,17,0,1


In [63]:
smookers_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 159256 entries, 0 to 159255
Data columns (total 23 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   age                  159256 non-null  int64  
 1   height(cm)           159256 non-null  int64  
 2   weight(kg)           159256 non-null  int64  
 3   waist(cm)            159256 non-null  float64
 4   eyesight(left)       159256 non-null  float64
 5   eyesight(right)      159256 non-null  float64
 6   hearing(left)        159256 non-null  int64  
 7   hearing(right)       159256 non-null  int64  
 8   systolic             159256 non-null  int64  
 9   relaxation           159256 non-null  int64  
 10  fasting blood sugar  159256 non-null  int64  
 11  Cholesterol          159256 non-null  int64  
 12  triglyceride         159256 non-null  int64  
 13  HDL                  159256 non-null  int64  
 14  LDL                  159256 non-null  int64  
 15  hemoglobin           1

In [64]:
smookers_df.isnull().sum()

age                    0
height(cm)             0
weight(kg)             0
waist(cm)              0
eyesight(left)         0
eyesight(right)        0
hearing(left)          0
hearing(right)         0
systolic               0
relaxation             0
fasting blood sugar    0
Cholesterol            0
triglyceride           0
HDL                    0
LDL                    0
hemoglobin             0
Urine protein          0
serum creatinine       0
AST                    0
ALT                    0
Gtp                    0
dental caries          0
smoking                0
dtype: int64

In [65]:
X_train = smookers_df.drop('smoking', axis=1)
y_train = smookers_df['smoking']

In [66]:
X_test = pd.read_csv(r'C:\Users\DAI.STUDENTSDC\Desktop\Machine Learning\Data Sets\Smookers\test.csv', index_col=0)
test_ans = pd.read_csv(r'C:\Users\DAI.STUDENTSDC\Desktop\Machine Learning\Data Sets\Smookers\sample_submission.csv', index_col=0)


---
Logistic Regression

In [67]:
ohe = OneHotEncoder(
    sparse_output=False, 
    drop='first'
).set_output(transform='pandas')

ct = make_column_transformer(
    ('passthrough', make_column_selector(dtype_exclude=object)),
    (ohe, make_column_selector(dtype_include=object)),
    verbose_feature_names_out=False,
).set_output(transform='pandas')

lr = LogisticRegression(random_state=24)

kfold = StratifiedKFold(n_splits=5, random_state=24, shuffle=True)

pipe = Pipeline([
    ("ct", ct), 
    ("lr", lr)
])

params = {
    "lr__solver": [
        "lbfgs",
        "liblinear",
        "newton-cg",
        "sag",
        "saga"
        "newton-cholesky"
    ],
    "lr__multi_class": [
        "ovr",
        "multinomial"
    ],
    "lr__C" : np.linspace(0.001, 10, 5),
}

gcv = GridSearchCV(
    estimator=pipe,
    param_grid=params,
    cv=kfold,
    verbose=2
)


gcv.fit(X_train, y_train)
 
print(gcv.best_params_)
print(gcv.best_score_)


 

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] END .lr__C=0.001, lr__multi_class=ovr, lr__solver=lbfgs; total time=   1.1s
[CV] END .lr__C=0.001, lr__multi_class=ovr, lr__solver=lbfgs; total time=   1.0s
[CV] END .lr__C=0.001, lr__multi_class=ovr, lr__solver=lbfgs; total time=   1.1s
[CV] END .lr__C=0.001, lr__multi_class=ovr, lr__solver=lbfgs; total time=   1.0s
[CV] END .lr__C=0.001, lr__multi_class=ovr, lr__solver=lbfgs; total time=   1.1s
[CV] END lr__C=0.001, lr__multi_class=ovr, lr__solver=liblinear; total time=   8.5s
[CV] END lr__C=0.001, lr__multi_class=ovr, lr__solver=liblinear; total time=   3.8s
[CV] END lr__C=0.001, lr__multi_class=ovr, lr__solver=liblinear; total time=   6.0s
[CV] END lr__C=0.001, lr__multi_class=ovr, lr__solver=liblinear; total time=   5.6s
[CV] END lr__C=0.001, lr__multi_class=ovr, lr__solver=liblinear; total time=   8.2s
[CV] END lr__C=0.001, lr__multi_class=ovr, lr__solver=newton-cg; total time=   3.1s
[CV] END lr__C=0.001, lr__mul

In [68]:
pd_cv = pd.DataFrame(gcv.cv_results_)
print(pd_cv.shape)


best_model = gcv.best_estimator_
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)




(50, 16)


In [70]:
y_pred

array([1, 0, 1, ..., 1, 0, 0], dtype=int64)

In [ ]:
sumbmission_df = test_ans.copy()
sumbmission_df['smoking'] = y_pred
sumbmission_df.to_csv('submission1.csv')



        smoking
id             
159256        1
159257        0
159258        1
159259        0
159260        1
...         ...
265422        0
265423        1
265424        1
265425        0
265426        0

[106171 rows x 1 columns]


---
Naive Bayes - Gaussian - Kernel

In [69]:
ohe = OneHotEncoder(
    sparse_output=False, 
    drop='first', 
    handle_unknown='ignore'
).set_output(transform='pandas')

ct = make_column_transformer(
    ('passthrough', make_column_selector(dtype_exclude=object)),
    (ohe, make_column_selector(dtype_include=object))
)

nb = GaussianNB()

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=24)

pipe = Pipeline([
    ('CT', ct), 
    ('NB', nb)
])



gcv = GridSearchCV(
    estimator=pipe, 
    cv=kfold,
    scoring='roc_auc',
    param_grid={ 
        'NB__var_smoothing': np.linspace(0.001, 0.1, 3),
    }, 

)

gcv.fit(X_train, y_train)
print('ROC AUC Score: ',gcv.best_score_)
y_pred = gcv.best_estimator_.predict(X_test)


 

ROC AUC Score:  0.7921370463657407
